#### **Задание 1** ####

Используйте файл с оценками фильмов `ml-latest-small/ratings.csv.` Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца `timestamp` для данного значения `userId.`

In [74]:
import pandas as pd

df_films = pd.read_csv('ml-latest-small/ratings.csv')

df_pivot = df_films.pivot_table(index = 'userId',  values = 'rating', aggfunc = 'count', fill_value = 0)

df_filter = df_pivot.loc[lambda row: row.rating >= 100]

df_max = df_films.pivot_table(index = 'userId',  values = 'timestamp', aggfunc = 'max', fill_value = 0)
df_max.rename(columns={'timestamp': 'timestamp_max'}, inplace=True)

df_min  = df_films.pivot_table(index = 'userId',  values = 'timestamp', aggfunc = 'min', fill_value = 0)
df_min.rename(columns={'timestamp': 'timestamp_min'}, inplace=True)

df_union = pd.concat([df_max, df_min, df_filter], axis=1)
# df_union_f = df_union.loc[lambda row: row.rating >= 100]
# df_union_fs
df_union['delta'] = df_union['timestamp_max'] - df_union['timestamp_min']
df_union.head()
df_union_f = df_union.loc[lambda row: row.rating >= 100]
df_union_f
avg = df_union_f['delta'].mean()
print(f'Среднее время жизни пользователей, выставивших более 100 оценок {avg}')

Среднее время жизни пользователей, выставивших более 100 оценок 39331748.56653992


#### **Задание 2** ####

   Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:

* таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
* аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:

* rzd - железнодорожные перевозки
* auto - автомобильные перевозки
* air - воздушные перевозки
* client_base - адреса клиентов



In [95]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

#------------------------------------------------------------------------
rzd['type'] = 'rzd_revenue'
auto['type'] = 'auto_revenue'
air['type'] = 'air_revenue'

df_union = pd.concat([rzd, auto, air], axis=0)
df_cl_rev = df_union[['client_id', 'type']]
print(df_cl_rev)
df_cl_rev.merge(client_base, how='left')

   client_id          type
0        111   rzd_revenue
1        112   rzd_revenue
2        113   rzd_revenue
3        114   rzd_revenue
4        115   rzd_revenue
0        113  auto_revenue
1        114  auto_revenue
2        115  auto_revenue
3        116  auto_revenue
4        117  auto_revenue
0        115   air_revenue
1        116   air_revenue
2        117   air_revenue
3        118   air_revenue


,client_id,type,address
0,111,rzd_revenue,Комсомольская 4
1,112,rzd_revenue,Энтузиастов 8а
2,113,rzd_revenue,Левобережная 1а
3,114,rzd_revenue,Мира 14
4,115,rzd_revenue,ЗЖБИиДК 1
5,113,auto_revenue,Левобережная 1а
6,114,auto_revenue,Мира 14
7,115,auto_revenue,ЗЖБИиДК 1
8,116,auto_revenue,Строителей 18
9,117,auto_revenue,Панфиловская 33


#### **Задание 3** ####

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого `user_id` известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

* У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
* Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
* Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.



#### таблица визитов:

visits
|user_id | visit | buy|           
|--------|-------|----|                  
| 11     | 20    | 300|                   
| 234    | 33    | 400|

#### таблица координат
coordinates
|user_id | latitude | longitude |
|--------|----------|-----------|
| 11     |  22,33300| 33,8890   | 
| 11     |  26,16656| 36,8478   | 
| 11     |  50,12300| 20,5677   | 
| 11     |  15,35570| 30,7999   | 
| 234    |  28,33655| 50,6788   | 
| 234    |  33,37890| 12,7899   | 

visits.join(coordinates, how='left')